# Render Hints: XML and Header Formatting

Learn how to use render hints to add XML wrappers and Markdown headers to your prompts.

In [2]:
from t_prompts import dedent, prompt

## XML Hint: Wrapping Content in Tags

The `xml=<tag>` render hint wraps interpolated content in XML-style tags. This is useful for:
- Structuring content for Claude's extended thinking format
- Creating well-formatted XML or HTML output
- Clearly delineating sections in prompts

In [3]:
# Basic XML wrapping
content = "This is my reasoning process for solving the problem."
p = prompt(t"{content:reasoning:xml=thinking}")

print(str(p))

<thinking>
This is my reasoning process for solving the problem.
</thinking>


In [4]:
# XML wrapping with multiline content
analysis = """Step 1: Understand the problem
Step 2: Break it down
Step 3: Solve each part"""

p = prompt(t"{analysis:steps:xml=analysis}")
print(str(p))

<analysis>
Step 1: Understand the problem
Step 2: Break it down
Step 3: Solve each part
</analysis>


In [5]:
# XML wrapping with nested prompts
inner_content = "Inner reasoning"
inner = prompt(t"{inner_content:ic:xml=step}")

outer_content = "Outer context"
outer = prompt(t"{outer_content:oc}\n{inner:inner:xml=process}")

print(str(outer))

Outer context
<process>
<step>
Inner reasoning
</step>
</process>


## Header Hint: Adding Markdown Headers

The `header=<heading>` or `header` render hint adds Markdown-style headers to your content. Headers automatically adjust their level based on nesting depth.

In [6]:
# Basic header with explicit text
task = "Translate the following text to French"
p = prompt(t"{task:t:header=Task Description}")

print(str(p))

# Task Description
Translate the following text to French


In [7]:
# Header using key as heading
instructions = "Follow these guidelines carefully"
p = prompt(t"{instructions:Instructions:header}")

print(str(p))

# Instructions
Follow these guidelines carefully


## Header Level Nesting

Headers automatically increment their level when nested through prompts that have the `header` hint. This creates proper Markdown hierarchy.

In [8]:
# Create nested structure with headers
subsection_content = "This is the subsection content"
subsection = prompt(t"{subsection_content:content:header=Subsection}")

section_content = "This is the section introduction"
section = prompt(t"{section_content:intro:header=Main Section}\n{subsection:sub:header}")

print(str(section))
print("\nNotice how headers increment: # Main Section, # sub, ## Subsection")

# Main Section
This is the section introduction
# sub
## Subsection
This is the subsection content

Notice how headers increment: # Main Section, # sub, ## Subsection


In [9]:
# Deep nesting with max level capping
level3 = prompt(t"{'Level 3 content':c:header=Deep Section}")
level2 = prompt(t"{level3:l3:header=Middle Section}")
level1 = prompt(t"{level2:l2:header=Top Section}")

print(str(level1))
print("\nHeader levels: # Top Section, ## Middle Section, ### Deep Section")

# Top Section
## Middle Section
### Deep Section
Level 3 content

Header levels: # Top Section, ## Middle Section, ### Deep Section


## Combining XML and Header Hints

You can use both hints together. The header will be rendered outside the XML wrapper.

In [10]:
# Combine header and XML
reasoning = "First, I'll analyze the problem. Then I'll propose a solution."
p = prompt(t"{reasoning:r:header=Analysis:xml=thinking}")

print(str(p))

# Analysis
<thinking>
First, I'll analyze the problem. Then I'll propose a solution.
</thinking>


In [11]:
# Complex nested example with both hints
inner_reasoning = "Detailed analysis of the sub-problem"
inner = prompt(t"{inner_reasoning:ir:header=Sub-Analysis:xml=step}")

outer_reasoning = "Overall approach to the problem"
outer = prompt(t"{outer_reasoning:or:header=Main Analysis}\n{inner:inner:header:xml=process}")

print(str(outer))

# Main Analysis
Overall approach to the problem
# inner
<process>
## Sub-Analysis
<step>
Detailed analysis of the sub-problem
</step>
</process>


## Using Hints with Lists

Render hints can wrap entire list interpolations, creating structured sections.

In [12]:
# XML wrapping a list
examples = [
    prompt(t"{ex:example:xml=example}")
    for ex in ["Example 1: Basic case", "Example 2: Edge case", "Example 3: Complex case"]
]

p = prompt(t"{examples:exs:xml=examples}")
print(str(p))

<examples>
<example>
Example 1: Basic case
</example>
<example>
Example 2: Edge case
</example>
<example>
Example 3: Complex case
</example>
</examples>


In [13]:
# Header for a list section
steps = [
    prompt(t"{step:s}")
    for step in ["Initialize variables", "Process data", "Return results"]
]

p = prompt(t"{steps:steps:header=Implementation Steps}")
print(str(p))

# Implementation Steps
Initialize variables
Process data
Return results


In [14]:
# Both hints on a list
items = [
    prompt(t"{item:i}")
    for item in ["First item", "Second item", "Third item"]
]

p = prompt(t"{items:items:header=Items:xml=list}")
print(str(p))

# Items
<list>
First item
Second item
Third item
</list>


## Combining with Separator Hints

You can use `sep=` along with `xml=` and `header=` hints to control list formatting.

In [15]:
# Custom separator with XML wrapper
items = [
    prompt(t"{item:i}")
    for item in ["apple", "banana", "cherry"]
]

p = prompt(t"{items:fruits:xml=fruits:sep=, }")
print(str(p))

<fruits>
apple, banana, cherry
</fruits>


In [16]:
# All three hints together
points = [
    prompt(t"{pt:p}")
    for pt in ["Point A", "Point B", "Point C"]
]

p = prompt(t"{points:points:header=Key Points:xml=list:sep= | }")
print(str(p))

# Key Points
<list>
Point A | Point B | Point C
</list>


## Integration with Dedenting

Render hints work seamlessly with dedented prompts for clean, readable source code.

In [17]:
# Header with dedenting
task = "Analyze the code for potential bugs"
p = dedent(t"""
    {task:task:header=Task}

    Please provide a detailed analysis.
    """)

print(str(p))

# Task
Analyze the code for potential bugs

Please provide a detailed analysis.


In [18]:
# XML with dedenting
reasoning = "This is my thought process"
p = dedent(t"""
    First, let me think:

    {reasoning:r:xml=thinking}

    Now I'll provide my answer.
    """)

print(str(p))

First, let me think:

<thinking>
This is my thought process
</thinking>

Now I'll provide my answer.


In [19]:
# Both hints with dedenting
analysis = "Step 1: Identify the problem\nStep 2: Propose solutions"
p = dedent(t"""
    {analysis:a:header=Analysis:xml=process}

    Conclusion: The problem is now clear.
    """)

print(str(p))

# Analysis
<process>
Step 1: Identify the problem
Step 2: Propose solutions
</process>

Conclusion: The problem is now clear.


## Realistic Example: Structured LLM Prompt

Build a well-structured prompt for Claude with multiple sections using headers and XML.

In [20]:
# Create structured sections
system_instructions = dedent(t"""
    You are an expert Python developer.
    Provide clear, well-commented code.
    """)

task_description = "Write a function to validate email addresses using regex"

examples = [
    prompt(t"Input: {inp:i}\nExpected: {out:o}")
    for inp, out in [
        ("user@example.com", "Valid"),
        ("invalid.email", "Invalid"),
        ("test@domain.co.uk", "Valid")
    ]
]

constraints = "Must handle international domains and subdomains"

# Assemble the full prompt
full_prompt = dedent(t"""
    {system_instructions:sys:header=System}

    {task_description:task:header=Task}

    {examples:exs:header=Examples:xml=examples}

    {constraints:constraints:header=Constraints:xml=requirements}

    Please provide your implementation below.
    """)

print(str(full_prompt))

# System
You are an expert Python developer.
Provide clear, well-commented code.

# Task
Write a function to validate email addresses using regex

# Examples
<examples>
Input: user@example.com
Expected: Valid
Input: invalid.email
Expected: Invalid
Input: test@domain.co.uk
Expected: Valid
</examples>

# Constraints
<requirements>
Must handle international domains and subdomains
</requirements>

Please provide your implementation below.


## Controlling Max Header Level

You can control the maximum header level using the `max_header_level` parameter in `render()`.

In [21]:
# Create deeply nested headers
level4 = prompt(t"{'Content 4':c:header=Level 4}")
level3 = prompt(t"{level4:l4:header=Level 3}")
level2 = prompt(t"{level3:l3:header=Level 2}")
level1 = prompt(t"{level2:l2:header=Level 1}")

# Default max level (4)
print("Default max_header_level=4:")
print(str(level1))
print()

# Custom max level (2)
print("Custom max_header_level=2:")
rendered = level1.render(max_header_level=2)
print(rendered.text)

Default max_header_level=4:
# Level 1
## Level 2
### Level 3
#### Level 4
Content 4

Custom max_header_level=2:
# Level 1
## Level 2
## Level 3
## Level 4
Content 4


## Provenance with Render Hints

Render hints are preserved in the prompt's provenance data.

In [22]:
import json

content = "Some content"
p = prompt(t"{content:c:header=Section:xml=data}")

# Export provenance
prov = p.to_provenance()
print(json.dumps(prov, indent=2))

print("\nRender hints are preserved:", prov['nodes'][0]['render_hints'])

{
  "strings": [
    "",
    ""
  ],
  "nodes": [
    {
      "key": "c",
      "expression": "content",
      "conversion": null,
      "format_spec": "c:header=Section:xml=data",
      "render_hints": "header=Section:xml=data",
      "index": 1,
      "value": "Some content"
    }
  ]
}

Render hints are preserved: header=Section:xml=data


## Best Practices

1. **Use XML for structure**: Wrap thinking, analysis, or specific content types in XML tags
2. **Use headers for sections**: Add headers to major sections of your prompts for clarity
3. **Combine thoughtfully**: Both hints together work well for major sections with structured content
4. **Consider nesting**: Headers automatically adjust levels - design your nesting hierarchy intentionally
5. **Use with dedenting**: Write readable source code with proper indentation and dedenting
6. **Max header level**: Keep header nesting reasonable (default max of 4 works well)
7. **XML tag names**: Use semantic tag names like `thinking`, `analysis`, `examples`